# Notebook pour le calcul des séries temporelles VAI (LANDSAT/S2)
### Des composites mensuelles sont calculées à partir des décades, puis les statistiques spatiales sont ensuites estimées

### Initialisation contexte test appli

In [ ]:
import os

WRK_DIR = os.path.normpath('/media/archive/EO4DM')
os.chdir(WRK_DIR)
# WRK_DIR = os.path.join('Y:/EO4DM')

TERRITORY = 'New Caledonia (Fr)'
month_start = '1999-09-01'
month_end = '2023-12-31'
TILE = '82075'
PRODUCT = 'VAI'
PERIOD = 'DECADE'
areas_key = 'nom'
CLEAN_RUNFOLDER = 0

TERRITORY_str = TERRITORY.replace(' ', '_').replace('(', '').replace(')', '')
DATA_HISTO = os.path.join(WRK_DIR,'DATA_HISTO',TERRITORY_str)
ANNEX_DIR = os.path.join(WRK_DIR,'ANNEX',TERRITORY_str)
INDIR_DATA = os.path.join(DATA_HISTO,'1_INDICATEURS','LOCAL',PERIOD)

In [ ]:
import glob
import shutil
import rasterio
import numpy as np
import pandas as pd
from tqdm import tqdm
import dmpipeline.ALERT_Processing.ALERT_main_functions as alertmain
import dmpipeline.GEOSTATS_Processing.GEOSTATS_processing_functions as geostats

### Prépare le dossier de sortie

In [ ]:
# --- Generate directories/sub-directories ---
OUTDIR_STATS = os.path.join(WRK_DIR, f'TIME_SERIES_LOCAL_DROUGHT_{TERRITORY_str}')
os.umask(0) # used to reset the directories permission
if not os.path.exists(OUTDIR_STATS):
    os.makedirs(OUTDIR_STATS)
    os.chmod(OUTDIR_STATS, 0o777)
elif int(CLEAN_RUNFOLDER)==1:
    shutil.rmtree(OUTDIR_STATS)
    os.makedirs(OUTDIR_STATS)
    os.chmod(OUTDIR_STATS, 0o777)


### Prépare données d'entrée (sat, masque)

In [ ]:
in_files = glob.glob(os.path.join(INDIR_DATA, f'{PRODUCT}_*{TILE}*.tif'))

# --- Prepare input masks and look-up table (for estimating geostats) ---
mask_ok = len(glob.glob(os.path.join(OUTDIR_STATS,'mask_Areas_*.tif')))>0
if mask_ok==0:
    file_areas = glob.glob(os.path.join(ANNEX_DIR, 'Areas', '*.shp'))
    file_landcover = glob.glob(os.path.join(ANNEX_DIR, 'Landcover', '*.tif'))
if len(file_areas)==0:
    print('Drought spatial stats will not be estimated : missing input shapefile containing geometries/areas to identify')
else:
    file_areas = file_areas[0]
    if len(file_landcover)!=0:
        file_landcover = file_landcover[0]
        go_landcover=1
    else: go_landcover=0
    geostats.prepareGeoStatsMasks(in_files[0], file_areas, OUTDIR_STATS, file_landcover=file_landcover, areas_key=areas_key)
    with rasterio.open(glob.glob(os.path.join(OUTDIR_STATS,'mask_Areas.tif'))[0]) as area_ds:
           maskAREA = area_ds.read(1)
           mask = (maskAREA != 0)
    area_lut = pd.read_csv(glob.glob(os.path.join(OUTDIR_STATS,'*.csv'))[0], sep=';')
    if go_landcover==1:
        with rasterio.open(glob.glob(os.path.join(OUTDIR_STATS,'mask_Areas_NOTrees_NOBuild.tif'))[0]) as NOTrees_ds, \
            rasterio.open(glob.glob(os.path.join(OUTDIR_STATS,'mask_Areas_Trees.tif'))[0]) as Trees_ds :
            mask_NOTrees_NOBuild = NOTrees_ds.read(1)
            mask_Trees = Trees_ds.read(1)

# --- AND Verify if output stats dataframes already exist (yes -> do not add hearder in csv file) ---
if PERIOD=='MONTH':
    stats_ok = len(glob.glob(os.path.join(OUTDIR_STATS, f'{PRODUCT}_STATS_M*.csv')))>0
elif PERIOD=='DECADE':
    stats_ok = len(glob.glob(os.path.join(OUTDIR_STATS, f'{PRODUCT}_STATS_D*.csv')))>0


### Calcul Geo Statistiques

In [ ]:
# /!\ Foncton pour le calcul des GeoStats : adaptée pour prendre en compte le cas où les "Sub-areas" dépassent les limites des tuiles landsat
#    -> qscore = np.nanmean (au lieu de np.mean pour éviter d'avoir un qscore à nan si dépassement)

def extractGeoStats(data, qscore, date, mask, maskAREA, area_df, territory, mask_NOTrees_NOBuild=None, mask_Trees=None):
    '''
    Spatial statistics are estimated on the entire Territory and on every Sub-Areas (predefined in input masks).
    Statistics are min, max and std of indices.
    The quality score is also given, corresponding to the pourcentage of good data used per pixel.
    Three dataframes are given :
        - considering any landcover
        - removing Dense Vegetation and Build Areas (_NoTrees_NoBuild)
        - keeping only Dense Vegetation (_Trees)

    Note : (sub)-areas on which to compute drought statistics are defined in spatial masks (.tif)
           The pixels of spatial masks are filled with specific values that are associated to areas names in the input look-up table (area_df)
           Masks and look-pu table are generated from 
    '''
    
    GeoStats_df = pd.DataFrame(columns=['LOCATION','DATE','MEAN','MIN','MAX','STD','QSCORE'])
    GeoStats_df_NOTrees_NOBuild = GeoStats_df.copy()
    GeoStats_df_Trees = GeoStats_df.copy()

    # --- Statistics over TERRITORY ---
    data_mean = np.nanmean(data[mask==1])
    data_min = np.nanmin(data[mask==1])
    data_max = np.nanmax(data[mask==1])
    data_std = np.nanstd(data[mask==1])
    data_qscore = np.nanmean(qscore[mask==1])
    d = {'LOCATION':territory,'DATE':date,'MEAN':data_mean,'MIN':data_min,
            'MAX':data_max,'STD':data_std,'QSCORE':data_qscore}
    GeoStats_df = pd.concat([GeoStats_df, pd.DataFrame([d])], ignore_index=True)
    del data_mean,data_min,data_max,data_std,data_qscore,d
    
    if mask_NOTrees_NOBuild is not None:
        data_mean = np.nanmean(data[mask_NOTrees_NOBuild==1])
        data_min = np.nanmin(data[mask_NOTrees_NOBuild==1])
        data_max = np.nanmax(data[mask_NOTrees_NOBuild==1])
        data_std = np.nanstd(data[mask_NOTrees_NOBuild==1])
        data_qscore = np.nanmean(qscore[mask_NOTrees_NOBuild==1])
        d = {'LOCATION':territory,'DATE':date,'MEAN':data_mean,'MIN':data_min,
                'MAX':data_max,'STD':data_std,'QSCORE':data_qscore}
        GeoStats_df_NOTrees_NOBuild = pd.concat([GeoStats_df_NOTrees_NOBuild, pd.DataFrame([d])], ignore_index=True)
        del data_mean,data_min,data_max,data_std,data_qscore,d
    
    if mask_Trees is not None:
        data_mean = np.nanmean(data[mask_Trees==1])
        data_min = np.nanmin(data[mask_Trees==1])
        data_max = np.nanmax(data[mask_Trees==1])
        data_std = np.nanstd(data[mask_Trees==1])
        data_qscore = np.nanmean(qscore[mask_Trees==1])
        d = {'LOCATION':territory,'DATE':date,'MEAN':data_mean,'MIN':data_min,
                'MAX':data_max,'STD':data_std,'QSCORE':data_qscore}
        GeoStats_df_Trees = pd.concat([GeoStats_df_Trees, pd.DataFrame([d])], ignore_index=True)
        del data_mean,data_min,data_max,data_std,data_qscore,d
    
    # --- Statistics over SUB-AREAS ---
    for a in area_df['nom']:
        
        ind_a = area_df.loc[area_df['nom']==a, 'OBJECTID']
        mask_a = (maskAREA == ind_a.values[0])
        Nb_ALLDATA_a = np.sum(mask_a)
        
        # If no pixels on commune: go to next iteration
        if Nb_ALLDATA_a==0:
            continue
        
        if Nb_ALLDATA_a!=0:
            data_mean_a = np.nanmean(data[mask_a==1])
            data_min_a = np.nanmin(data[mask_a==1])
            data_max_a = np.nanmax(data[mask_a==1])
            data_std_a = np.nanstd(data[mask_a==1])
            data_qscore_a = np.nanmean(qscore[mask_a==1])
            d_a = {'LOCATION':a,'DATE':date,'MEAN':data_mean_a,'MIN':data_min_a,
                    'MAX':data_max_a,'STD':data_std_a,'QSCORE':data_qscore_a}
            GeoStats_df = pd.concat([GeoStats_df, pd.DataFrame([d_a])], ignore_index=True)
            del data_mean_a,data_min_a,data_max_a,data_std_a,data_qscore_a,d_a
        
        if mask_NOTrees_NOBuild is not None:
            mask_a_NOTrees_NOBuild = (mask_a==1) & (mask_NOTrees_NOBuild==1)
            Nb_ALLDATA_a_NOTrees_NOBuild = np.sum(mask_a_NOTrees_NOBuild)

            if Nb_ALLDATA_a_NOTrees_NOBuild!=0:                   
                data_mean_a = np.nanmean(data[mask_a_NOTrees_NOBuild==1])
                data_min_a = np.nanmin(data[mask_a_NOTrees_NOBuild==1])
                data_max_a = np.nanmax(data[mask_a_NOTrees_NOBuild==1])
                data_std_a = np.nanstd(data[mask_a_NOTrees_NOBuild==1])
                data_qscore_a = np.nanmean(qscore[mask_a_NOTrees_NOBuild==1])
                d_a = {'LOCATION':a,'DATE':date,'MEAN':data_mean_a,'MIN':data_min_a,
                        'MAX':data_max_a,'STD':data_std_a,'QSCORE':data_qscore_a}
                GeoStats_df_NOTrees_NOBuild = pd.concat([GeoStats_df_NOTrees_NOBuild, pd.DataFrame([d_a])], ignore_index=True)
                del data_mean_a,data_min_a,data_max_a,data_std_a,data_qscore_a,d_a
        
        if mask_Trees is not None:
            mask_a_Trees = (mask_a==1) & (mask_Trees==1)
            Nb_ALLDATA_a_Trees = np.sum(mask_a_Trees)

            if Nb_ALLDATA_a_Trees!=0:                   
                data_mean_a = np.nanmean(data[mask_a_Trees==1])
                data_min_a = np.nanmin(data[mask_a_Trees==1])
                data_max_a = np.nanmax(data[mask_a_Trees==1])
                data_std_a = np.nanstd(data[mask_a_Trees==1])
                data_qscore_a = np.nanmean(qscore[mask_a_Trees==1])
                d_a = {'LOCATION':a,'DATE':date,'MEAN':data_mean_a,'MIN':data_min_a,
                        'MAX':data_max_a,'STD':data_std_a,'QSCORE':data_qscore_a}
                GeoStats_df_Trees = pd.concat([GeoStats_df_Trees, pd.DataFrame([d_a])], ignore_index=True)
                del data_mean_a,data_min_a,data_max_a,data_std_a,data_qscore_a,d_a
        
    return GeoStats_df, GeoStats_df_NOTrees_NOBuild, GeoStats_df_Trees

In [ ]:
date_vect_dt = pd.date_range(start=month_start, end=month_end, freq='M')
date_vect_str = date_vect_dt.strftime("%Y%m")

with rasterio.open(in_files[0]) as d_ds:
    profile_like = d_ds.profile

count_head = 0 # the first time, add header to geostats data frame


for date_str in tqdm(date_vect_str):

    # --- Collect decades of same month and aggregate to monthly period ---
    
    files_m = glob.glob(os.path.join(INDIR_DATA, f'{PRODUCT}_*{TILE}_{date_str}*.tif'))

    if len(files_m)==0:
        continue

    elif len(files_m)==1:
        in_f = files_m[0]
        with rasterio.open(in_f) as in_ds:
            profile_in = in_ds.profile      
            DATA = in_ds.read(1)
            if profile_in['count']==2:
                DATA_QSCORE = in_ds.read(2)
            else:
                DATA_QSCORE = ~np.isnan(DATA)
    
    elif len(files_m)>1:

        N_files_m = len(files_m)
        DATA_MAT = np.full((profile_like['height'],profile_like['width'],N_files_m), np.nan, 'float32')
        DATA_MAT_QSCORE = np.full_like(DATA_MAT, np.nan, 'float32')
        for i in range(N_files_m):
            with rasterio.open(files_m[i]) as d_ds:
                DATA_MAT[:,:,i] = d_ds.read(1)
                if d_ds.count==2:
                    DATA_MAT_QSCORE[:,:,i] = d_ds.read(2)
                elif d_ds.count==1:
                    DATA_MAT_QSCORE[:,:,i] = ~np.isnan(DATA_MAT[:,:,i])
        DATA = np.nanmean(DATA_MAT,axis=2)
        DATA_QSCORE = np.nanmean(DATA_MAT_QSCORE,axis=2)


   # --- Estimate spatial stats ---
    date_df = pd.to_datetime(date_str, format='%Y%m')
    
    if go_landcover==1:
        GeoStats_df, GeoStats_df_NOTrees_NOBuild, GeoStats_df_Trees = extractGeoStats(DATA, DATA_QSCORE, date_df, mask, maskAREA,
                                                                                      area_lut, TERRITORY, mask_NOTrees_NOBuild, mask_Trees)
        GeoStats_df_NOTrees_NOBuild = GeoStats_df_NOTrees_NOBuild.sort_values(by=['LOCATION','DATE'])
        GeoStats_df_Trees = GeoStats_df_Trees.sort_values(by=['LOCATION','DATE'])

        GeoStats_df_NOTrees_NOBuild.to_csv(
            os.path.join(OUTDIR_STATS, f'{PRODUCT}_STATS_M_NoTrees_NoBuild.csv'),
            index = False,
            float_format='%.2f',
            decimal = '.',
            sep = ';',
            mode='a',
            header = (count_head==0 and stats_ok==0))
        GeoStats_df_Trees.to_csv(
            os.path.join(OUTDIR_STATS, f'{PRODUCT}_STATS_M_Trees.csv'),
            index = False,
            float_format='%.2f',
            decimal = '.',
            sep = ';',
            mode='a',
            header = (count_head==0 and stats_ok==0))
        
        del GeoStats_df_NOTrees_NOBuild, GeoStats_df_Trees

    else:
        GeoStats_df, _, _ = extractGeoStats(DATA, DATA_QSCORE, date_df, mask, maskAREA, area_lut, TERRITORY)
    
    GeoStats_df = GeoStats_df.sort_values(by=['LOCATION','DATE'])

    GeoStats_df.to_csv(os.path.join(OUTDIR_STATS, f'{PRODUCT}_STATS_M.csv'),
        index = False,
        float_format='%.2f',
        decimal = '.',
        sep = ';',
        mode='a',
        header = (count_head==0 and stats_ok==0))
    
    count_head += 1
    
    del GeoStats_df, date_df, DATA, DATA_QSCORE